In [1]:
print("too many bitches, where the fellas at?")

too many bitches, where the fellas at?


In [2]:
import os
import netket as nk
from netket.operator.spin import sigmax, sigmaz, sigmay
# from src.wilson import * 
from src.models import * 
import matplotlib.pyplot as plt

In [3]:
L = 8
g = nk.graph.Square(L, pbc=True)
print(f'g.n_edges = {g.n_edges}, LxLx2 = {L**2 * 2}')
hi = nk.hilbert.Spin(s=1/2, N=g.n_edges)

g.n_edges = 128, LxLx2 = 128


In [4]:
gerbil = GERBIL(graph=g)
sampler = nk.sampler.MetropolisLocal(hi)
vstate = nk.vqs.MCState(sampler, model=gerbil, n_samples=1008)
# optimizer = nk.optimizer.Sgd(learning_rate=0.05)
# preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
# vmc = nk.driver.VMC(h, optimizer, variational_state=vstate, preconditioner=preconditioner)
# log = nk.logging.RuntimeLog()
# vmc.run(n_iter=10, out=log)

Shapes of rbm_output and v_bias: (0, 8) (8,)


ValueError: Incompatible shapes for broadcasting: shapes=[(0, 8), (0,)]